# Doğal Dil İşleme + Karar Bilimi (Natural Language Processing + Decision Science)

👩🏻‍🏫 Bu görevde şunları bir araya getireceğiz:
* 🗣 Doğal Dil İşleme (Natural Language Processing)
* 📊 Karar Bilimi (Decision Science)

🎯 Amaç, Olist üzerindeki ürünlerin ve satıcıların **olumsuz (kötü) yorumlarını** anlamaktır.

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
# Data Manipulation
import numpy as np
import pandas as pd
pd.set_option("display.max_columns",None)

# Machine Learning
from sklearn.pipeline import make_pipeline

# Language Processing
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import nltk
import string
import unidecode as unidecode

# Vectorizers and NLP Models
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation

🕵🏻‍♂️ Olist’in CEO’su [Tiago Dalvi](https://www.linkedin.com/in/tiagodalvi/)’nin senden yorumları okuyup anlamanı istediğini hayal et.

- Müşteriler siparişlerini **1**, **2** veya **3** puanla değerlendirdiklerinde ne söylediler?
- En sık karşılaşılan olumsuz yorumlar neler?
    - En kötü puanlanan ürünler hakkında?
    - En kötü puanlanan satıcılar hakkında?


## (0) Kurulum 🔨

Öncelikle, Olist incelemeleriyle ilgili tüm bilgileri içeren DataFrame'i yükleyeceğiz!

In [4]:
df = pd.read_csv("https://d32aokrjazspmn.cloudfront.net/materials/olist_reviews.csv")

In [5]:
df.head()

,review_id,length_review,review_score,order_id,product_category_name,full_review
0,e64fb393e7b32834bb789ff8bb30750e,37,5,658677c97b385a9be170737859d3511b,ferramentas_jardim,Recebi bem antes do prazo estipulado.
1,f7c4243c7fe1938f181bec41a392bdeb,100,5,8e6bfb81e283fa7e4f11123a3fb894f1,esporte_lazer,Parabéns lojas lannister adorei comprar pela ...
2,8670d52e15e00043ae7de4c01cc2fe06,174,4,b9bf720beb4ab3728760088589c62129,eletroportateis,recomendo aparelho eficiente. no site a marca ...
3,4b49719c8a200003f700d3d986ea1a19,45,4,9d6f15f95d01e79bd1349cc208361f09,beleza_saude,"Mas um pouco ,travando...pelo valor ta Boa.\r\n"
4,3948b09f7c818e2d86c9a546758b2335,56,5,e51478e7e277a83743b6f9991dbfa3fb,informatica_acessorios,"Super recomendo Vendedor confiável, produto ok..."


## (2) Metin Temizleme (Text Cleaning)

🧹 `cleaning(sentence)` işlevini oluşturun ve yorumlara uygulayın. **NLTK'da Portekizce lemmatizer bulunmadığını unutmayın** (genellikle bulunmaz, ancak `nltk.stem.RSLPStemmer` kök bulucu vardır).

In [6]:
import re
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer

stop_words = set(stopwords.words('portuguese'))
stemmer = SnowballStemmer("portuguese")

def cleaning(sentence):
    # unicode normalize
    sentence = unidecode.unidecode(sentence)
    
    # küçük harf
    sentence = sentence.lower()
    
    # sayıları kaldır
    sentence = re.sub(r'\d+', '', sentence)
    
    # noktalama işaretlerini kaldır
    sentence = sentence.translate(str.maketrans('', '', string.punctuation))
    
    # tokenization
    tokens = word_tokenize(sentence)
    
    # stopwords kaldır + stemming
    tokens = [
        stemmer.stem(word)
        for word in tokens
        if word not in stop_words
    ]
    
    return ' '.join(tokens)

In [7]:
df['clean_review'] = df['full_review'].apply(cleaning)

## (3) Kötü yorumların analizi

### (3.1) Düşük inceleme puanlarına sahip veri kümesi

😱 1 ile 3 arasında puan alan yorumların oranı nedir? 

In [8]:
low_score_ratio = (df['review_score'].between(1, 3).mean()) * 100
round(low_score_ratio, 2)

26.72

🕵🏻‍♂️ Bu yorumlara odaklanalım...

In [9]:
bad_reviews = df[df['review_score'].between(1, 3)].copy()
bad_reviews.shape

(9658, 7)

### (3.2) Vektörleştirme

🔡 ➡️ 🔢 Metinlerini vektörleştir.

- **Bigram**’leri (iki kelimelik ifadeler) mutlaka hesaba kat.
- Çok sık geçen kelimeleri çıkarmak için `max_df = 0.75` ayarla.
- Spoiler uyarısı: Sonunda **20.000+** kelimeye ulaşacaksın…  
  Bu challenge için sadece `max_features = 5000` ile sınırla.

In [10]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(
    ngram_range=(1, 2),
    max_df=0.75,
    max_features=5000
)

X = vectorizer.fit_transform(bad_reviews['clean_review'])
X.shape

(9658, 5000)

### (3.3) LDA

🕵🏻‍♂️ LDA'yı uyarlayın:
- `n_components = 3` seçin
- *.transform()* ile Konuların Belge Karışımını gösterin
- *.components_* ile Konu Karışımını gösterin

In [11]:
from sklearn.decomposition import LatentDirichletAllocation

lda = LatentDirichletAllocation(n_components=3, random_state=42)
lda.fit(X)

LatentDirichletAllocation(n_components=3, random_state=42)

#### Belge Karışımı (Konular için)

In [12]:
doc_topic = lda.transform(X)
doc_topic[:5]

array([[0.17585755, 0.80002943, 0.02411303],
       [0.95051802, 0.02540729, 0.02407469],
       [0.02037481, 0.02052187, 0.95910332],
       [0.04785868, 0.04942163, 0.90271969],
       [0.08540222, 0.83065465, 0.08394313]])

👉 Her inceleme için en önemli konuyu rapor edelim

In [13]:
bad_reviews['dominant_topic'] = doc_topic.argmax(axis=1)
bad_reviews[['review_score', 'product_category_name', 'dominant_topic']].head()

,review_score,product_category_name,dominant_topic
14,1,eletronicos,1
23,3,ferramentas_jardim,0
24,3,beleza_saude,2
25,1,moveis_decoracao,2
28,3,esporte_lazer,1


#### Konu Karışımı (Kelimeler için)

In [14]:
lda.components_.shape

(3, 5000)

#### Konular

🎁 Size bazı yardımcı fonksiyonlar sağladık:
- `topic_word`: Tek bir konu (topic) için en önemli kelimeleri ve ağırlıklarını döndürür
- `print_topics`: LDA tarafından bulunan farklı konuları, en önemli kelimeleriyle birlikte yazdırır

In [15]:
def topic_word(vectorizer, model, topic, topwords, with_weights = True):
    topwords_indexes = topic.argsort()[:-topwords - 1:-1]
    if with_weights == True:
        topwords = [(vectorizer.get_feature_names_out()[i], round(topic[i],2)) for i in topwords_indexes]
    if with_weights == False:
        topwords = [vectorizer.get_feature_names_out()[i] for i in topwords_indexes]
    return topwords

In [16]:
def print_topics(vectorizer, model, topwords):
    for idx, topic in enumerate(model.components_):
        print("-"*20)
        print("Topic %d:" % (idx))
        print(topic_word(vectorizer, model, topic, topwords))


🕵🏻‍♂️ Konuları en çok kullanılan kelimelerle birlikte yazdırın:

In [17]:
print_topics(vectorizer, lda, topwords=10)

--------------------
Topic 0:
[('compr', 1436.15), ('receb', 1134.07), ('so', 989.2), ('entreg', 773.28), ('ped', 757.86), ('nao', 637.63), ('apen', 611.83), ('vei', 533.33), ('dois', 495.61), ('cheg', 441.55)]
--------------------
Topic 1:
[('produt', 2977.75), ('nao', 2011.8), ('entreg', 1754.02), ('receb', 1659.5), ('compr', 807.16), ('nao receb', 768.43), ('receb produt', 687.7), ('praz', 641.64), ('aind', 602.65), ('vei', 465.25)]
--------------------
Topic 2:
[('nao', 3118.57), ('produt', 2508.37), ('vei', 1028.43), ('qualidad', 595.57), ('bom', 571.53), ('compr', 563.68), ('recom', 525.16), ('gost', 468.43), ('cheg', 385.28), ('produt nao', 341.76)]


🇧🇷 Burada biraz Brezilya Portekizcesi kelimeler var:
- _cadeiras = chairs_
- _produto = product_
- _recomendo = recommend (não recomendo == not recommend)_
- _bom = good_
- _comprei = bought_
- _veio = came_
- _errado = wrong_
- _gostaria = I would like to..._
- _duas = two_
- _nao = not_
- _entregue = delivered_
- _pecas = part_
- _ainda = yet_
- _recebi = received_

👉 Bir konuyla ilişkili en popüler kelimeleri göster

In [18]:
# Her topic için en önemli kelimeleri çıkar
topic_word_mixture = {}

feature_names = vectorizer.get_feature_names_out()

for topic_idx, topic in enumerate(lda.components_):
    top_words = [
        feature_names[i]
        for i in topic.argsort()[:-11:-1]
    ]
    topic_word_mixture[topic_idx] = top_words

In [19]:
bad_reviews['most_important_words'] = bad_reviews['dominant_topic'].apply(
    lambda i: topic_word_mixture[i]
)

In [21]:
bad_reviews[
    [
        "review_id",
        "review_score",
        "product_category_name",
        "clean_review",
        "dominant_topic",
        "most_important_words"
    ]
].head()

,review_id,review_score,product_category_name,clean_review,dominant_topic,most_important_words
14,e233e51d11511bf30e568c76360ace52,1,eletronicos,receb soment control mid split estil falt cont...,1,"[produt, nao, entreg, receb, compr, nao receb,..."
23,8b230a1373c6dc4bd867099fda1d7039,3,ferramentas_jardim,compr duas unidad so receb agor fac,0,"[compr, receb, so, entreg, ped, nao, apen, vei..."
24,cb2fc3e5711b5ae85e0491ee18af63ed,3,beleza_saude,produt bom por vei mim nao condiz fot anunci,2,"[nao, produt, vei, qualidad, bom, compr, recom..."
25,60c714ed14cef913944a3147094a4742,1,moveis_decoracao,produt inferior mal acab,2,"[nao, produt, vei, qualidad, bom, compr, recom..."
28,0bd4dcc4f6c4621baf37f73495cad8c4,3,esporte_lazer,entreg praz,1,"[produt, nao, entreg, receb, compr, nao receb,..."


## (3.4) Pipeline Tf-Idf ve LDA

In [22]:
from sklearn import set_config
set_config("diagram")

🔨 Önceki Vectorizer ve LDA'yı birbirine bağlayan bir Pipeline oluşturun.

Temizlenmiş metinlere uyarlayın.

In [23]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation

pipeline = Pipeline(
    steps=[
        (
            "tfidf",
            TfidfVectorizer(
                ngram_range=(1, 2),
                max_df=0.75,
                max_features=5000
            )
        ),
        (
            "lda",
            LatentDirichletAllocation(
                n_components=3,
                random_state=42
            )
        )
    ]
)

💡 `pipeline.components_` ile bileşenlere erişmeye çalışırsanız, Pipeline'da `components_` olmadığı için bu YÜRÜMEZ. Ancak, LDA'ya erişmek için `pipeline._final_estimator` kullanabilirsiniz. Ve buradan konulara erişebilirsiniz!

In [24]:
pipeline._final_estimator

LatentDirichletAllocation(n_components=3, random_state=42)

In [26]:
lda_pipeline_model = pipeline.named_steps["lda"]
lda_pipeline_model

LatentDirichletAllocation(n_components=3, random_state=42)

In [28]:
pipeline.fit(bad_reviews["clean_review"])

Pipeline(steps=[('tfidf',
                 TfidfVectorizer(max_df=0.75, max_features=5000,
                                 ngram_range=(1, 2))),
                ('lda',
                 LatentDirichletAllocation(n_components=3, random_state=42))])

Pipeline ile **Belge Karışımı**:

In [29]:
lda_pipeline_model = pipeline.named_steps["lda"]
lda_pipeline_model.components_

array([[ 1.72074571,  0.47727613,  0.34457903, ...,  1.3636634 ,
         5.69385952,  0.50891123],
       [ 2.72310181,  0.73038123,  0.39029875, ...,  0.36281301,
         0.7218864 ,  0.35857501],
       [ 0.46318268, 12.61290037,  2.57961145, ...,  1.79902642,
         2.45773434,  6.1465366 ]])

In [30]:
pipeline_doc_topics = pipeline.transform(bad_reviews["clean_review"])
pipeline_doc_topics[:5]

array([[0.1548353 , 0.76652414, 0.07864056],
       [0.84352313, 0.0801876 , 0.07628927],
       [0.07378246, 0.07234841, 0.85386913],
       [0.10099475, 0.1018926 , 0.79711265],
       [0.13581186, 0.15308178, 0.71110636]])

Pipeline ile **Konu Karışımı**:

In [31]:
tfidf_vectorizer = pipeline.named_steps["tfidf"]
feature_names = tfidf_vectorizer.get_feature_names_out()

for idx, topic in enumerate(lda_pipeline_model.components_):
    print(f"\nTopic {idx}:")
    print([feature_names[i] for i in topic.argsort()[:-11:-1]])


Topic 0:
['compr', 'receb', 'so', 'bom', 'entreg', 'apen', 'ped', 'dois', 'falt', 'nao']

Topic 1:
['produt', 'nao', 'receb', 'nao receb', 'entreg', 'receb produt', 'aind', 'demor', 'defeit', 'aind nao']

Topic 2:
['produt', 'nao', 'gost', 'vei', 'recom', 'qualidad', 'bom', 'cheg', 'praz', 'bem']


## (4) 🎁 Ürün Kategorileri

### (4.1) Ürün kategorilerine göre gruplandırma

📈 Veri kümesini `product_category_name` ile gruplandırın ve performanslarını inceleyin.

In [32]:
# Performansa göre ürün kategorileri - sayı, ortalama, medyan ve standart sapmaya bakın
product_categories = df.groupby(by = 'product_category_name').agg({
        'review_score': ["count", "mean", "median", "std"]
    })

# Analiz için belirli bir süreden daha az satılan ürünleri kaldırma
cutoff = 50
product_categories = product_categories[product_categories[("review_score", "count")] > cutoff]

# Ürün kategorilerini performansa göre sıralama
product_categories = product_categories.sort_values(by = [('review_score', 'mean'),
                                                          ('review_score', 'std')],
                                                    ascending = [False, True])
product_categories

review_score                   \
                                                      count      mean median   
product_category_name                                                          
alimentos_bebidas                                        75  4.560000    5.0   
livros_interesse_geral                                  152  4.539474    5.0   
malas_acessorios                                        375  4.317333    5.0   
livros_tecnicos                                          83  4.301205    5.0   
pcs                                                      67  4.283582    5.0   
alimentos                                               151  4.231788    5.0   
papelaria                                               758  4.201847    5.0   
instrumentos_musicais                                   228  4.192982    5.0   
fashion_calcados                                        101  4.188119    5.0   
fashion_bolsas_e_acessorios                             726  4.168044    5.0   
perfumaria                                             1094  4.167276    5.0   
construcao_ferramentas_jardim                            85  4.164706    5.0   
cool_stuff                                             1360  4.154412    5.0   
ferramentas_jardim                                     1391  4.105679    5.0   
beleza_saude                                           3046  4.104071    5.0   
brinquedos                                             1271  4.101495    5.0   
esporte_lazer                                          2637  4.097838    5.0   
bebidas                                                 111  4.090090    5.0   
pet_shop                                                603  4.086235    5.0   
construcao_ferramentas_iluminacao                        86  4.081395    5.0   
eletroportateis                                         242  4.061983    5.0   
automotivo                                             1533  4.049576    5.0   
eletrodomesticos                                        354  4.033898    5.0   
eletrodomesticos_2                                       89  4.011236    5.0   
relogios_presentes                                     2211  4.009950    5.0   
construcao_ferramentas_construcao                       277  4.007220    5.0   
consoles_games                                          339  4.005900    5.0   
utilidades_domesticas                                  2192  4.005018    5.0   
eletronicos                                             942  3.979830    5.0   
industria_comercio_e_negocios                            82  3.963415    5.0   
bebes                                                   882  3.955782    5.0   
sinalizacao_e_seguranca                                  61  3.934426    5.0   
telefonia                                              1654  3.874244    5.0   
cama_mesa_banho                                        3891  3.853765    5.0   
artes                                                    80  3.850000    5.0   
casa_conforto                                           175  3.817143    5.0   
moveis_decoracao                                       2311  3.815232    5.0   
moveis_sala                                             138  3.789855    5.0   
informatica_acessorios                                 2473  3.774767    5.0   
audio                                                   134  3.768657    5.0   
market_place                                            102  3.745098    4.0   
moveis_cozinha_area_de_servico_jantar_e_jardim           94  3.734043    4.0   
construcao_ferramentas_seguranca                         61  3.721311    5.0   
agro_industria_e_comercio                                57  3.701754    4.0   
telefonia_fixa                                           96  3.697917    4.0   
casa_construcao                                         177  3.672316    4.0   
climatizacao                                             93  3.655914    4.0   
fashion_roupa_masculina                        

### (4.2) En kötü ürün kategorileri

👎 *Ortalama değerlendirme puanı* açısından en kötü beş kategoriyi `worst_products` adlı bir değişkene kaydedin.

In [33]:
worst_products = product_categories.tail(5).sort_values(by = [("review_score", "count")],
                                                       ascending = False)
worst_products

review_score                           
                               count      mean median       std
product_category_name                                          
moveis_escritorio                546  3.298535    4.0  1.562366
casa_construcao                  177  3.672316    4.0  1.498094
telefonia_fixa                    96  3.697917    4.0  1.576938
climatizacao                      93  3.655914    4.0  1.625173
fashion_roupa_masculina           56  3.482143    4.0  1.737198

👇 Yalnızca `worst_products` öğelerini içeren bir `worst_products_review` DataFrame oluşturun.

In [36]:
# Worst ürün kategorilerinin review'larını al
worst_products_reviews = df[df.product_category_name.isin(worst_products.index)].copy()

# Pipeline ile belge-topic dağılımı -> baskın topic
topic_dist = pipeline.transform(worst_products_reviews["clean_review"])
worst_products_reviews["dominant_topic"] = topic_dist.argmax(axis=1)

# Her topic için en önemli kelimeler (topic_word_mixture hazır olmalı)
worst_products_reviews["most_important_words"] = worst_products_reviews["dominant_topic"].apply(
    lambda i: topic_word_mixture[i]
)

# İstenen kolonları göster
worst_products_reviews[
    [
        "review_id",
        "review_score",
        "product_category_name",
        "clean_review",
        "dominant_topic",
        "most_important_words",
    ]
].head()

,review_id,review_score,product_category_name,clean_review,dominant_topic,most_important_words
45,c422274e50e900b46fee429016c5458d,3,moveis_escritorio,gost,2,"[nao, produt, vei, qualidad, bom, compr, recom..."
126,5d6f9cddc8335878d8bf20c3bd4602e8,5,moveis_escritorio,meg recom receb produt corret dentr prazoverif...,1,"[produt, nao, entreg, receb, compr, nao receb,..."
212,1f836d160f50247a394034abd93b7c24,3,casa_construcao,esper instal,1,"[produt, nao, entreg, receb, compr, nao receb,..."
219,edd86e0af4dc62b340c60d846643cbf9,5,moveis_escritorio,produt ok praz estendidopor entreg praz divulg...,2,"[nao, produt, vei, qualidad, bom, compr, recom..."
232,e47020cf52099bce9dc57524ea41218b,4,moveis_escritorio,indic loj compr sempr lannist,1,"[produt, nao, entreg, receb, compr, nao receb,..."


### (4.3). En kötü ürünler için konular

❓ En kötü ürünlerin konuları nelerdir? ❓

In [38]:
worst_products_reviews["dominant_topic"].value_counts()

dominant_topic
2    506
1    255
0    207
Name: count, dtype: int64

In [40]:
bad_frequency = list(worst_products_reviews["dominant_topic"].value_counts().index)

In [41]:
[topic_word_mixture[i] for i in bad_frequency]

[['nao',
  'produt',
  'vei',
  'qualidad',
  'bom',
  'compr',
  'recom',
  'gost',
  'cheg',
  'produt nao'],
 ['produt',
  'nao',
  'entreg',
  'receb',
  'compr',
  'nao receb',
  'receb produt',
  'praz',
  'aind',
  'vei'],
 ['compr',
  'receb',
  'so',
  'entreg',
  'ped',
  'nao',
  'apen',
  'vei',
  'dois',
  'cheg']]

## (5) 🎁 Satıcılar...

* En kötü satıcılar tarafından ne tür ürünler satıldı?
* En kötü satıcılar için başlıca yorumlar nelerdir?

### (5.1) En kötü satıcılar

In [42]:
from olist.seller import Seller
sellers = Seller().get_training_data()
sellers.columns

Index(['seller_id', 'seller_city', 'seller_state', 'delay_to_carrier',
       'wait_time', 'date_first_sale', 'date_last_sale', 'months_on_olist',
       'n_orders', 'quantity', 'quantity_per_order', 'sales', 'review_score',
       'share_of_five_stars', 'share_of_one_stars', 'revenues',
       'cost_of_reviews', 'profits'],
      dtype='object')

👇 En kötü satan 10 ürünü seçin ve bunları `worst_sellers` adlı bir değişkene kaydedin.

In [43]:
worst_sellers = sellers[["seller_id", "review_score", "profits"]].sort_values(
    by = "profits",
    ascending = True).head(10)
worst_sellers

,seller_id,review_score,profits
769,6560211a19b47992c3666cc44a7e94c0,3.937093,-9274.245850
315,8b321bb669392f5163d04c59e235e066,4.074547,-8593.715345
946,ea8482cd71df3c1969d7b9473ff13abc,3.997371,-8379.215572
1358,cc419e0650a3c5ba77189a1882b7556a,4.077586,-7856.445356
2689,1835b56ce799e6a4dc4eddc053f04066,3.637441,-4426.195114
2360,4a3ca9315b744ce9f8e9374361493884,3.827873,-4324.612498
2744,1ca7077d890b907f89be8c954a02686a,2.333333,-4259.351772
1214,d2374cbcbb3ca4ab1086534108cc3ab7,3.751880,-4237.307248
453,1f50f920176fa81dab994f9023523100,4.130342,-3746.079000
1066,2eb70248d66e0e3ef83659f71b244378,2.693467,-3457.742015


### (5.2) En kötü satıcılar tarafından satılan ürünler

In [45]:
from olist.product import Product

In [46]:
products = Product().get_training_data() [["product_id", "category"]]
products

,product_id,category
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumery
1,3aa071139cb16b67ca9e5dea641aaa2f,art
2,96bd76ec8810374ed1b65e291975717f,sports_leisure
3,cef67bcfe19066a932b7673e239eb23d,baby
4,9dc1a7de274444849c219cff195d0b71,housewares
...,...,...
31479,a0b7d5a992ccda646f2d34e418fff5a0,furniture_decor
31480,bf4538d88321d0fd4412a93c974510e6,construction_tools_lights
31481,9a7c6041fa9592d9d9ef6cfe62a71f8c,bed_bath_table
31482,83808703fc0706a22e264b9d75f04a2e,computers_accessories


❓ En kötü satıcılar tarafından satılan ürün türleri nelerdir? ❓

In [50]:
worst_categories = (
    worst_products_reviews["product_category_name"]
    .value_counts()
)

worst_categories

product_category_name
moveis_escritorio          546
casa_construcao            177
telefonia_fixa              96
climatizacao                93
fashion_roupa_masculina     56
Name: count, dtype: int64

### (5.3) En kötü satıcılar için kategoriler ve konular

🎁 İşte bazı kullanışlı işlevler:
- Bir satıcı tarafından satılan ürün kategorilerini göstermek için `focus_seller(seller_id)`
- Bir satıcı için en sık kullanılan konuların en popüler kelimelerini göstermek için `bad_reviews_seller`

In [53]:
def focus_seller(seller_id):
    return sellers_product_category[sellers_product_category.seller_id == seller_id].value_counts()

In [54]:
bad_reviews_sellers = worst_products_reviews.merge(data["order_items"])
bad_reviews_sellers.head(3)

NameError: name 'data' is not defined

In [ ]:
def bad_reviews_seller(bad_reviews_sellers, seller_id):
    mask = (bad_reviews_sellers.seller_id == seller_id)
    temp = bad_reviews_sellers[mask]
    if len(temp) > 0: # satıcı kötü yorumlar veri çerçevesinde görünüyorsa
        most_frequent_topic_seller = list(temp.most_important_topic.value_counts().head(1).index)[0]
        return topic_word_mixture[most_frequent_topic_seller]

❓Bu en az satan ürünlerin her biri için en sık kullanılan ürün kategorilerini ve kelimeleri gösterin ❓

In [ ]:
for worst_seller in worst_sellers["seller_id"]:
    print("-"*50)
    print(f"Focusing on the seller #{worst_seller}...")
    print(focus_seller(worst_seller))
    print(bad_reviews_seller(bad_reviews_sellers, worst_seller))


🏁 Tebrikler. NLP'nin bazı temellerini (Ön İşleme + Vektörleştirme + NB/LDA) öğrendiniz ve bu yeni “uzmanlığı” Karar Bilimi ile birleştirdik.

💾 `git add / commit / push` yapmayı unutmayın.